In [12]:
import numpy as np

# can be 'PIN' or 'ROLLER'
class Joint(object):
    def __init__(self, id=0, x=0, y=0):
        self.id=id
        self.x=x
        self.y=y

def calculate_distance(start_joint, end_joint):
    dx = end_joint.x - start_joint.x
    dy = end_joint.y - start_joint.y
    return np.abs(dx*dx + dy*dy)
# list of joints: dictionary {ID : {Joint}  }, 
# list of edges: tuples (A, B)    
    
class Truss(object):
    def __init__(self, edgeList=[], joints=[], fixed_joint='', roller_joint='', center_joint=''):
        self.edgeList = edgeList
        self.joints = joints
        self.fixed_joint = fixed_joint
        self.roller_joint = roller_joint
        self.center_joint = center_joint
        self.member_cross_area = 19.625e-4 #m^2
        self.member_density = 8050 #kg / m^3
        self.joint_mass = 5 #kg
        
        self.jointDict = {}
        for joint in self.joints:
            self.jointDict[joint.id] = joint
    
    def calculate_mass(self):
        total_length = 0
        num_joints = len(self.joints)
        for edge in self.edgeList:
            joint_1 = self.jointDict[edge[0]]
            joint_2 = self.jointDict[edge[1]]
            length = calculate_distance(joint_1, joint_2)
            total_length += length
        mass = total_length * self.member_cross_area * self.member_density
        mass += num_joints * self.joint_mass
        return mass

In [13]:
# truss solving function:
# takes in:
# TRUSS object (list of nodes, list of edges)
# list of forces
# AppliedForces : (JointID, magnitude, direction (in y direction))
def solve_forces(appliedForces, truss):
    numAppliedForces = len(appliedForces)
    numJoints = len(truss.joints)
    numReactions = 3 # 2 (Fx, Fy) at fixed joint, 1 (Fy) at roller
    numEdges = len(truss.edgeList)
    numEquations = numAppliedForces + numEdges + numReactions
    M = np.zeros((numEquations, numEquations))
    #M system of equations to solve [<applied forces> .... <x forces in joints> <y forces in joints> ]
    #F : vector of forces to solve for [<applied forces>, <forces in beams>, <reactions>]
    
    E = np.zeros((numEquations, 1)) #solution to system of equations M*F = E
    #populate rows for  E 
    for i in range(numAppliedForces):
        E[i] = appliedForces[i][1]
        M[i][i] = 1

    i = 0 
    for i in range(numJoints):
        joint = truss.joints[i]
        jointId = joint.id
        row = i*2 + numAppliedForces
        for j in range(len(appliedForces)):
            col = j
            if(appliedForces[j][0] == jointId):
                M[row][col] = 1 # Force:(jointId, magnitude)
        
        for j in range(len(truss.edgeList)):
            col = j+numAppliedForces
            edge = truss.edgeList[j]
            otherJointId = None
            if(edge[0] == jointId):
                otherJointId = edge[1]
            elif(edge[1] == jointId):
                otherJointId = edge[0]
            if(otherJointId != None):
                otherJoint = truss.jointDict[otherJointId]
                x1 = joint.x
                y1 = joint.y
                x2 = otherJoint.x
                y2 = otherJoint.y
                dx = x2 - x1
                dy = y2 - y1
                
                theta = np.abs(np.arctan(dy/dx))
                x_multiplier = np.sign(dx) * np.cos(theta)
                y_multiplier = np.sign(dy) * np.sin(theta)
                
                M[row][col] = y_multiplier
                M[row+1][col] = x_multiplier
        if(joint.id == truss.fixed_joint):
            M[row][numEquations - 3] = 1 # reaction, assume in positive direction
            M[row + 1][numEquations - 2] = 1 #y reaction, assume in positive direction
        if(joint.id == truss.roller_joint):
            M[row][numEquations - 1] = 1
    
    F = np.linalg.solve(M, E)
    F = np.around(F,2)
    return M, E, F

In [14]:
#joint_x_list = [0, 0.5, 1]
#joint_y_list = [0, 0.866, 0]
#jointIds = ['A', 'B', 'C']
joint_x_list = [0, 1, 2, 3, 4]
joint_y_list = [0, 1, 0, 1, 0]
jointIds = ['A', 'B', 'C', 'D', 'E']
jointList = []
for i in range(len(jointIds)):
    joint = Joint(jointIds[i],joint_x_list[i],joint_y_list[i]);
    jointList.append(joint)

edge_list = [('A', 'B'), ('B', 'C'), ('A', 'D'), ('D', 'E'), ('D', 'B'), ('E','C'), ('B', 'E')]

truss = Truss(edge_list, jointList, 'A', 'E')
mass = truss.calculate_mass()
print(mass)

562.13625


In [15]:
applied_force_list = [('B', -100)] # 500 Newtons
M, E, F = solve_forces(applied_force_list, truss)
print(M)
print(F)

1
3
5
7
9
[[ 1.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.70710678  0.          0.31622777  0.          0.          0.
   0.          1.          0.          0.        ]
 [ 0.          0.70710678  0.          0.9486833   0.          0.          0.
   0.          0.          1.          0.        ]
 [ 1.         -0.70710678 -0.70710678  0.          0.          0.          0.
  -0.31622777  0.          0.          0.        ]
 [ 0.         -0.70710678  0.70710678  0.          0.          1.          0.
   0.9486833   0.          0.          0.        ]
 [ 0.          0.          0.70710678  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.70710678  0.          0.          0.          1.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.31622777 -0.70710678  0.          0.
   0.    

In [27]:
import operator

def sort_by_x(list_of_nodes):
    list_of_nodes.sort(key = operator.attrgetter('x'))
    
def getKey(item):
    return item[1]

def refine(x, y, std):
    x_refined = np.random.normal(x, std)
    y_refined = np.random.normal(y, std)
    return np.round(x_refined,2), np.round(y_refined,2)

def sorted_list_of_nodes_by_distances(joint_list, start_joint):
    distance_list = []
    for joint in joint_list:
        if(joint.id == start_joint.id):
            continue
        r = calculate_distance(start_joint, joint)
        distance_list.append((joint.id, r))
    return sorted(distance_list, key = getKey)

class TrussGenerator:
    def __init__(self, width, max_height, min_height, min_edge_length):
        self.width = width
        self.x_min = -width/2
        self.x_max = width/2
        self.max_height = max_height
        self.min_height = min_height
        self.min_edge_length = min_edge_length
        self.ascii_offset = 65
        self.joint_list = []
        self.dict_of_symmetric_joints = {}
        self.center_joint_id = ''
        self.pin_joint_id = ''
        self.roller_joint_id = ''
        self.list_of_edges = []
    
    def add_joint_to_list(self, joint_list, x, y):
        index = len(joint_list)
        joint = Joint(chr(index + self.ascii_offset), x, y)
        joint_list.append(joint)
        return joint.id

    def add_symmetric_pair_of_joints(self, id1, id2):
        self.dict_of_symmetric_joints[id1] = id2
        self.dict_of_symmetric_joints[id2] = id1

    def generate_nodes(self, num_joints):
        joint_list = []
        index = 0
        x_end = width/2
        x_start = -width/2
        # place fixed joints 
        self.pin_joint_id = self.add_joint_to_list(joint_list, x_start, 0)
        self.roller_joint_id = self.add_joint_to_list(joint_list, x_end, 0)
        self.add_symmetric_pair_of_joints(self.pin_joint_id, self.roller_joint_id)
        # place central joints
        is_even = False
        num_central_joints = 1
        if(num_joints % 2 == 0):
            is_even = True
            num_central_joints = 2
        #place first central joint
        y_coarse = np.random.uniform(self.min_height,self.max_height)
        y_refined = np.random.normal(y_coarse, min_edge_length/2)
        self.center_joint_id = self.add_joint_to_list(joint_list, 0,y_refined)
        self.add_symmetric_pair_of_joints(self.center_joint_id, self.center_joint_id)
        if(is_even):
            y_coarse_2 = np.random.uniform(min_height, max_height)
            if(y_coarse_2 == y_coarse):
                y_coarse_2 = (y_coarse + 1) % (self.max_height - self.min_height) + self.min_height
            y_refined_2 = np.random.normal(y_coarse, min_edge_length/2)
            center2_id = self.add_joint_to_list(joint_list, 0,y_refined_2)
            self.add_symmetric_pair_of_joints(self.center2_id, self.center2_id)

        min_length = self.min_edge_length
        x_grid_size = (self.width / 2) / self.min_edge_length - 2
        y_grid_size = (self.max_height - self.min_height) / self.min_edge_length       
        
        total_positions = x_grid_size * y_grid_size
        if(total_positions < num_joints):
            min_length = min_length / 2
        
        #generate list of tuples with all possible positions
        possible_position_list = []
        for x in np.arange(min_length, width/2 - min_length, min_length):
            for y in np.arange(min_height, max_height, min_length):
                possible_position_list.append((x, y))
        
        num_remaining_joints = int((num_joints - 2 - num_central_joints) / 2)
        std = self.min_edge_length / 4
        for i in range(num_remaining_joints):
            list_size = len(possible_position_list)
            index = np.random.randint(0,list_size)
            x = possible_position_list[index][0]
            y = possible_position_list[index][1]
            x, y = refine(x,y, std)
            id1 = self.add_joint_to_list(joint_list, x, y)
            possible_position_list.pop(index)
            # mirror the joint
            x_mirrored = -x
            id2 = self.add_joint_to_list(joint_list, x_mirrored, y)
            self.add_symmetric_pair_of_joints(id1, id2)

        self.joint_list = joint_list
        return joint_list
    
    def connect_nodes(self, id1, id2):
        self.connected_node_table[id1].append(id2)
        self.connected_node_table[id2].append(id1)
        edge = (id1, id2)
        self.list_of_edges.append(edge)

    def build_oonnections(self):
        sort_by_x(joint_list)
        
        self.list_of_edges = []
        self.connected_node_table = {}
        #initialize table with empty lists
        for joint in joint_list:
            self.connected_node_table[joint.id] = []
        
        num_nodes = len(joint_list)
        num_desired_connections = 2*num_nodes - 3
        if(num_desired_connections < 3):
            return
        even_num = False
        if(num_nodes % 2 == 0):
            even_num = True
        
        index_start = 0
        index_end = index_end = int((num_nodes - 1)/2)
        if(even_num):
            index_end = int(num_nodes - 2)/2
        
        index_center_node_start = index_end
        index_center_node_end = int(index_end + 1)
        if(even_num):
            index_center_node_end = int(index_center_node_start + 2)
        
        num_connections = 0
        subset_of_nodes = self.joint_list[0:index_center_node_end]
        
        
        if(even_num): #connect 2 center joints
            joint1_id = self.joint_list[index_center_node_start].id
            joint2_id = self.joint_list[index_center_node_start+1].id
            self.connect_nodes(joint1_id, joint2_id)
        else: #connect 2 joints closest to center joint
            joint1_id = self.joint_list[index_center_node_start - 1].id
            joint2_id = self.dict_of_symmetric_joints[joint1_id]
            self.connect_nodes(joint1_id, joint2_id)

        for node in subset_of_nodes:
            list_of_nodes_by_distance = sorted_list_of_nodes_by_distances(subset_of_nodes, node)
            closest_node_id = list_of_nodes_by_distance[0][0]
            next_closest_node_id = list_of_nodes_by_distance[1][0]
            # check if closest node already connected
            if(closest_node_id not in self.connected_node_table[node.id]):
                self.connect_nodes(node.id, closest_node_id)
                #mirror the connection
                mirror_id_1 = self.dict_of_symmetric_joints[node.id]
                mirror_id_2 = self.dict_of_symmetric_joints[closest_node_id]
                self.connect_nodes(mirror_id_1, mirror_id_2)
            # check if next closest node is already connected 
            if(next_closest_node_id not in self.connected_node_table[node.id]):
                self.connect_nodes(node.id, next_closest_node_id)
                #mirror the connection
                mirror_id_1 = self.dict_of_symmetric_joints[node.id]
                mirror_id_2 = self.dict_of_symmetric_joints[next_closest_node_id]
                self.connect_nodes(mirror_id_1, mirror_id_2)
        
        succesful = (len(self.list_of_edges) == num_desired_connections) 

        return self.list_of_edges, self.connected_node_table, succesful

In [28]:
# test truss generator
width = 4
max_height = 1
min_height = 0
min_edge_length = 0.5
truss_geneator = TrussGenerator(width, max_height, min_height, min_edge_length)
joint_list = truss_geneator.generate_nodes(7)
#list_of_connections, connected_node_table, succesful = build_oonnections(list_of_joints, dict_of_symmetric_joints, center_joint_id, pin_joint_id)
for joint in joint_list:
    print(joint.id, joint.x, joint.y)


A -2.0 0
B 2.0 0
C 0 0.6165021660491152
D 0.74 0.65
E -0.74 0.65
F 0.55 0.09
G -0.55 0.09


In [31]:
# test for build_connections function
#[('A', 'B'), ('B', 'C'), ('A', 'D'), ('D', 'E'), ('D', 'B'), ('E','C'), ('B', 'E')]
list_of_joints = jointList
dict_of_symmetric_joints = {'A':'E', 'B':'D', 'C':'C', 'D':'B', 'E':'A'}
center_joint_id = 'C'
pin_joint_id = 'A'
list_of_connections, connected_node_table, succesful = truss_geneator.build_oonnections()
print(succesful)
print("connections \n", list_of_connections)
print("table \n", connected_node_table)

True
connections 
 [('G', 'F'), ('A', 'E'), ('B', 'D'), ('A', 'G'), ('B', 'F'), ('E', 'G'), ('D', 'F'), ('E', 'C'), ('D', 'C'), ('G', 'C'), ('F', 'C')]
table 
 {'A': ['E', 'G'], 'E': ['A', 'G', 'C'], 'G': ['F', 'A', 'E', 'C'], 'C': ['E', 'D', 'G', 'F'], 'F': ['G', 'B', 'D', 'C'], 'D': ['B', 'F', 'C'], 'B': ['D', 'F']}


In [34]:
def evaluate_fitness(truss):
    center_joint = truss.center_joint
    default_force = -100 #kN
    default_member_strength = 490 # kN, for radius of 2.5 cm
    
    applied_forces = [(center_joint, default_force)]
    M, E, F = solve_forces(applied_forces, truss)
    max_force_in_member = F.argmax()
    
    safety_factor = default_member_strength / max_force_in_member
    mass = truss.calculate_mass()
    # maybe should calculate maximum mass?
    fitness = safety_factor / mass
    return fitness

In [36]:
# test fitness function
fitness = evaluate_fitness(truss)
print(fitness)

1
3
5
7
9
0.871674794145
